In [ ]:
#Daily Feature
import pandas as pd
import numpy as np

In [ ]:
def candle_based_feature(stock_daily_data):

    print('weekly candles features')
    feature_groups = stock_daily_data.groupby('stock_name')

    def candle_based_function(feature_group):
        df = feature_group
        initial_index = feature_group.index
        df = feature_group
        df.index = [pd.Timestamp(item) for item in df['date']]
        return_df = pd.DataFrame({\
            'max_price7d': df['adj_max_price'].rolling(5).max(),\
            'min_price7d': df['adj_min_price'].rolling(5).min(),\
            'first_price7d': df['adj_first_price'].rolling(5).agg(lambda rows: rows[0]),\
            'last_price7d': df['adj_last_price'].rolling(5).agg(lambda rows: rows[-1]),\
            'yesterday_price7d': df['yesterday_price'].rolling(5).agg(lambda rows: rows[0]),\
            'max_price14d': df['adj_max_price'].rolling(10).max(),\
            'min_price14d': df['adj_min_price'].rolling(10).min(),\
            'first_price14d': df['adj_first_price'].rolling(10).agg(lambda rows: rows[0]),\
            'last_price14d': df['adj_last_price'].rolling(10).agg(lambda rows: rows[-1]),\
            'yesterday_price14d': df['yesterday_price'].rolling(10).agg(lambda rows: rows[0]),\
            'max_price21d': df['adj_max_price'].rolling(15).max(),\
            'min_price21d': df['adj_min_price'].rolling(15).min(),\
            'first_price21d': df['adj_first_price'].rolling(15).agg(lambda rows: rows[0]),\
            'last_price21d': df['adj_last_price'].rolling(15).agg(lambda rows: rows[-1]),\
            'yesterday_price21d': df['yesterday_price'].rolling(15).agg(lambda rows: rows[0]),\
            'max_price30d': df['adj_max_price'].rolling(23).max(),\
            'min_price30d': df['adj_min_price'].rolling(23).min(),\
            'first_price30d': df['adj_first_price'].rolling(23).agg(lambda rows: rows[0]),\
            'last_price30d': df['adj_last_price'].rolling(23).agg(lambda rows: rows[-1]),\
            'yesterday_price30d': df['yesterday_price'].rolling(23).agg(lambda rows: rows[0]),\
            'max_price60d': df['adj_max_price'].rolling(44).max(),\
            'min_price60d': df['adj_min_price'].rolling(44).min(),\
            'first_price60d': df['adj_first_price'].rolling(44).agg(lambda rows: rows[0]),\
            'last_price60d': df['adj_last_price'].rolling(44).agg(lambda rows: rows[-1]),\
            'yesterday_price60d': df['yesterday_price'].rolling(44).agg(lambda rows: rows[0]),\
            'max_price90d': df['adj_max_price'].rolling(70).max(),\
            'min_price90d': df['adj_min_price'].rolling(70).min(),\
            'first_price90d': df['adj_first_price'].rolling(70).agg(lambda rows: rows[0]),\
            'last_price90d': df['adj_last_price'].rolling(70).agg(lambda rows: rows[-1]),\
            'yesterday_price90d': df['yesterday_price'].rolling(70).agg(lambda rows: rows[0]),\
            })
        return_df.index = initial_index
        return return_df

    candle_based_data = feature_groups.apply(candle_based_function).reset_index(drop=True)

    candle_based_detail = {\
            'max_price7d': {'description':'max price in 7-day-candle.'},\
            'min_price7d': {'description':'min price in 7-day-candle.'},\
            'first_price7d': {'description':'first price in 7-day-candle.'},\
            'last_price7d': {'description':'last price in 7-day-candle.'},\
            'yesterday_price14d': {'description':'yesterday price in 14-day-candle.'},\
            'max_price14d': {'description':'max price in 14-day-candle.'},\
            'min_price14d': {'description':'min price in 14-day-candle.'},\
            'first_price14d': {'description':'first price in 14-day-candle.'},\
            'last_price14d': {'description':'last price in 14-day-candle.'},\
            'yesterday_price14d': {'description':'yesterday price in 14-day-candle.'},\
            'max_price21d': {'description':'max price in 21-day-candle.'},\
            'min_price21d': {'description':'min price in 21-day-candle.'},\
            'first_price21d': {'description':'first price in 21-day-candle.'},\
            'last_price21d': {'description':'last price in 21-day-candle.'},\
            'yesterday_price21d': {'description':'yesterday price in 21-day-candle.'},\
            'max_price30d': {'description':'max price in 30-day-candle.'},\
            'min_price30d': {'description':'min price in 30-day-candle.'},\
            'first_price30d': {'description':'first price in 30-day-candle.'},\
            'last_price30d': {'description':'last price in 30-day-candle.'},\
            'yesterday_price30d': {'description':'yesterday price in 30-day-candle.'},\
            'max_price60d': {'description':'max price in 60-day-candle.'},\
            'min_price60d': {'description':'min price in 60-day-candle.'},\
            'first_price60d': {'description':'first price in 60-day-candle.'},\
            'last_price60d': {'description':'last price in 60-day-candle.'},\
            'yesterday_price60d': {'description':'yesterday price in 60-day-candle.'},\
            'max_price90d': {'description':'max price in 90-day-candle.'},\
            'min_price90d': {'description':'min price in 90-day-candle.'},\
            'first_price90d': {'description':'first price in 90-day-candle.'},\
            'last_price90d': {'description':'last price in 90-day-candle.'},\
            'yesterday_price90d': {'description':'yesterday price in 90-day-candle.'},\
            }

    price_data = pd.concat([stock_daily_data,candle_based_data],axis=1)

    feature_groups = price_data.groupby('stock_name')

    def shadow_based_featurs(feature_group):
        df = feature_group
        initial_index = feature_group.index
        df = feature_group
        df.index = [pd.Timestamp(item) for item in df['date']]
        
        return_df = pd.DataFrame({\
            'shadow_up7d': (df['max_price7d'] - np.maximum(df['first_price7d'],df['last_price7d']))/
                                (df['yesterday_price7d']),\
            'shadow_low7d': (df['min_price7d'] - np.minimum(df['first_price7d'],df['last_price7d']))/
                                (df['yesterday_price7d']),\
            'body7d': np.abs(df['last_price7d']-df['first_price7d'])/df['yesterday_price7d'],\
            'shadow_up14d': (df['max_price14d'] - np.maximum(df['first_price14d'],df['last_price14d']))/
                                (df['yesterday_price14d']),\
            'shadow_low14d': (df['min_price14d'] - np.minimum(df['first_price14d'],df['last_price14d']))/
                                (df['yesterday_price14d']),\
            'body14d': np.abs(df['last_price14d']-df['first_price14d'])/df['yesterday_price14d'],\
            'shadow_up21d': (df['max_price21d'] - np.maximum(df['first_price21d'],df['last_price21d']))/
                                (df['yesterday_price21d']),\
            'shadow_low21d': (df['min_price21d'] - np.minimum(df['first_price21d'],df['last_price21d']))/
                                (df['yesterday_price21d']),\
            'body21d': np.abs(df['last_price21d']-df['first_price21d'])/df['yesterday_price21d'],\
            'shadow_up30d': (df['max_price30d'] - np.maximum(df['first_price30d'],df['last_price30d']))/
                                (df['yesterday_price30d']),\
            'shadow_low30d': (df['min_price30d'] - np.minimum(df['first_price30d'],df['last_price30d']))/
                                (df['yesterday_price30d']),\
            'body30d': np.abs(df['last_price30d']-df['first_price30d'])/df['yesterday_price30d'],\
            'shadow_up60d': (df['max_price60d'] - np.maximum(df['first_price60d'],df['last_price60d']))/
                                (df['yesterday_price60d']),\
            'shadow_low60d': (df['min_price60d'] - np.minimum(df['first_price60d'],df['last_price60d']))/
                                (df['yesterday_price60d']),\
            'body60d': np.abs(df['last_price60d']-df['first_price60d'])/df['yesterday_price60d'],\
            'shadow_up90d': (df['max_price90d'] - np.maximum(df['first_price90d'],df['last_price90d']))/
                                (df['yesterday_price90d']),\
            'shadow_low90d': (df['min_price90d'] - np.minimum(df['first_price90d'],df['last_price90d']))/
                                (df['yesterday_price90d']),\
            'body90d': np.abs(df['last_price90d']-df['first_price90d'])/df['yesterday_price90d']\
            })
        return_df.index = initial_index
        return return_df

    shadow_based_data = feature_groups.apply(shadow_based_featurs).reset_index(drop=True) 

    shadow_based_detail = {\
            'shadow_up7d': {'description':'max price in 7-day-candle.'},\
            'shadow_low7d': {'description':'min price in 7-day-candle.'},\
            'body7d': {'description':'first price in 7-day-candle.'},\
            'shadow_up14d': {'description':'max price in 14-day-candle.'},\
            'shadow_low14d': {'description':'min price in 14-day-candle.'},\
            'body14d': {'description':'first price in 14-day-candle.'},\
            'shadow_up21d': {'description':'max price in 21-day-candle.'},\
            'shadow_low21d': {'description':'min price in 21-day-candle.'},\
            'body21d': {'description':'first price in 21-day-candle.'},\
            'shadow_up30d': {'description':'max price in 30-day-candle.'},\
            'shadow_low30d': {'description':'min price in 30-day-candle.'},\
            'body30d': {'description':'first price in 30-day-candle.'},\
            'shadow_up60d': {'description':'max price in 60-day-candle.'},\
            'shadow_low60d': {'description':'min price in 60-day-candle.'},\
            'body60d': {'description':'first price in 60-day-candle.'},\
            'shadow_up90d': {'description':'max price in 90-day-candle.'},\
            'shadow_low90d': {'description':'min price in 90-day-candle.'},\
            'body90d': {'description':'first price in 90-day-candle.'}\
    }
    

    feature_data = pd.concat([shadow_based_data, candle_based_data ], \
        axis=1)
    feature_detail = dict()
    for d in [candle_based_detail, shadow_based_detail]:
        feature_detail.update(d)

    return feature_data, feature_detail
